# Red neuronal para reproducir la funcion XNOR de 3 valores de entrada.

In [3]:
# Preparamos el entorno
%pylab
%matplotlib inline
%config InlineBackend.figure_format = 'retina'
# %config blas.ldflags = '-lblas'

import numpy as np
import theano
import theano.tensor as T
# print (T.config)


Using matplotlib backend: Qt5Agg
Populating the interactive namespace from numpy and matplotlib


RuntimeError: 
To use MKL 2018 with Theano either update the numpy conda packages to
their latest build or set "MKL_THREADING_LAYER=GNU" in your
environment.


In [188]:
# Definimos funciones utiles
def floatX(X):
    return np.asarray(X, dtype = theano.config.floatX)

def init_weights(shape):
    # Devuelve pesos aleatorios para una matriz con las filas y columnas 
    # pasadas como parametro
    return theano.shared(floatX(np.random.randn(*shape) * 0.01))

def modelo_RNXOR_3_2(entrada, salida, iteraciones):
    # Modelo de red neuronal de 3 neuronas a la entrada y dos a la salida,
    # para generar la funcion XOR
    
    # Matrices de pesos de las capas de entrada y salida y Bias
    w_input = init_weights((3,2))
    print(w_input.get_value())
    w_output = init_weights((2,1))
    print(w_output.get_value())
    b1 = theano.shared(1.)
    b2 = theano.shared(1.)
    
    # Definimos los valores de entrada y Salida
    X = T.fmatrix()
    Y = T.fmatrix()
    # Creamos la red neuronal. En primera capa con sigmoide, en segunda con softmax
    Sal_Capa1 = T.nnet.sigmoid(T.dot(X, w_input)+b1)
    Sal_CapaFin = T.nnet.softmax(T.dot(Sal_Capa1, w_output)+b2)
    # Nos quedamos con el valor con mayor probabilidad
    respuesta = T.argmax(Sal_CapaFin, axis=1)
    # respuesta = Sal_CapaFin[T.argmax(Sal_CapaFin)]
    print('----------------------------------------')
    print(respuesta.type, respuesta.ndim)
    print('----------------------------------------')
    print(respuesta.flatten)
    
    # Entrenamos la red neuronal, para lo que definimos una funcion de coste
    f_coste = T.nnet.binary_crossentropy(respuesta, Y).mean()
    # f_coste = T.mean(T.nnet.binary_crossentropy(Sal_CapaFin, Y))
    # Tambien una funcion de ajuste de pesos y bias para propagacion del error
    ajuste = T.grad(f_coste, [w_input, w_output] )
    print(*ajuste)
    aprendizaje = []
    for inp, aju in zip([w_input, w_output], ajuste):
        aprendizaje.append([inp, inp - aju * 0.01])
        
    print('aprendizaje:')
    for ind in range(len(aprendizaje)):
        print(*aprendizaje[ind])
    # Definimos la funcion de entrenamiento
    f_train = theano.function (
          inputs = [X,Y],
          outputs = [respuesta, f_coste] ,
          updates = aprendizaje,
          allow_input_downcast = True
        )
    # Definimos la funcion de ejecucion de la red:
    f_predict = theano.function (
          inputs= [X],
          outputs = respuesta,
          allow_input_downcast = True
        )
    
    # Iteramos hasta obtener el resultado:
    cost = []
    for i in range(iteraciones):
        pred, cost_iter = f_train(entrada, salida)
        # print(pred)
        cost.append(cost_iter)
    # Se imprimen los resultados por pantalla
    print('Los resultados de la red son:')
    for i in range(len(entrada)):
        print('El resultado para [%d, %d, %d] es %.2f' % (inputs[i][0], inputs[i][1], inputs[i][2], pred[i]))
    
    # Función de esfuerzo en función del número de iteraciones
    plt.plot(cost)


In [189]:
# Conjunto de datos de entrenamiento
inputs = [[0, 0, 0], [0, 0, 1], [0, 1, 0], [0, 1, 1], [1, 0, 0], [1, 0, 1], [1, 1, 0] , [1, 1, 1]]
outputs = [[0, 1, 1, 0, 1, 0, 0, 1]]

for ind in range(len(inputs)):
    print(inputs[ind])

modelo_RNXOR_3_2(inputs, outputs, 100)

[0, 0, 0]
[0, 0, 1]
[0, 1, 0]
[0, 1, 1]
[1, 0, 0]
[1, 0, 1]
[1, 1, 0]
[1, 1, 1]
[[-0.01944644  0.00354864]
 [-0.00397421  0.0007802 ]
 [-0.00741165 -0.01417206]]
[[ 0.00890333]
 [-0.01078799]]
----------------------------------------
TensorType(int64, vector) 1
----------------------------------------
<bound method _tensor_py_operators.flatten of argmax>
dot.0 dot.0
aprendizaje:
<TensorType(float64, matrix)> Elemwise{sub,no_inplace}.0
<TensorType(float64, matrix)> Elemwise{sub,no_inplace}.0


ValueError: Input dimension mis-match. (input[0].shape[0] = 1, input[1].shape[0] = 8)
Apply node that caused the error: Elemwise{Composite{((i0 * i1) + (i2 * log1p((-i3))))}}[(0, 1)](<TensorType(float32, matrix)>, LogSoftmax.0, Elemwise{sub,no_inplace}.0, Softmax.0)
Toposort index: 21
Inputs types: [TensorType(float32, matrix), TensorType(float64, matrix), TensorType(float32, matrix), TensorType(float64, matrix)]
Inputs shapes: [(1, 8), (8, 1), (1, 8), (8, 1)]
Inputs strides: [(32, 4), (8, 8), (32, 4), (8, 8)]
Inputs values: ['not shown', 'not shown', 'not shown', 'not shown']
Outputs clients: [[Sum{acc_dtype=float64}(Elemwise{Composite{((i0 * i1) + (i2 * log1p((-i3))))}}[(0, 1)].0)]]

HINT: Re-running with most Theano optimization disabled could give you a back-trace of when this node was created. This can be done with by setting the Theano flag 'optimizer=fast_compile'. If that does not work, Theano optimizations can be disabled with 'optimizer=None'.
HINT: Use the Theano flag 'excep